In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import netCDF4 as nc
import glob
from scipy.io import loadmat
from matplotlib.image import imread
import matplotlib.pyplot as plt
import os
from scipy import interpolate
import datetime
import scipy 
import pickle
import cartopy.crs as ccrs
import scipy.spatial as spatial
from combining_and_aligning import combining_files
from combining_and_aligning import lining_up_lines



In [ ]:
projection = ccrs.Stereographic(
    central_latitude=90.0,
    central_longitude=-45.0,
    false_easting=0.0,
    false_northing=0.0,
    true_scale_latitude=70.0,
    globe=ccrs.Globe('WGS84'))

In [ ]:
files = glob.glob('/scratch/icebridgedata/IRSNO1B_SnowRadar_Greenland_Repeat_Flights/20*/flat100/*mat')
files2 = []
for file in files:
    if 'deconv' not in file:
        files2.append(file)
    else:
        print('bad')
        
        
fvals = np.asarray(files2)
files_sorted = np.sort(fvals)



## Sort files

In [ ]:
midlons=[]
midlats=[]



for file in files_sorted:
    print(file)
    ds = loadmat(file)
    lat = ds['Latitude'][0]
    midlat = lat[int(len(lat)/2)]
    lon = ds['Longitude'][0]
    midlon = lon[int(len(lon)/2)]
    midlats.append(midlat)
    midlons.append(midlon)


pickle.dump({'Midlons':midlons,'midlats':midlats,'files':np.asarray(files_sorted)},open('Locations_of_Greenland_Box_Echograms_S.p','wb'))


        

In [ ]:
stuff = pickle.load(open('/data/users/mdattler/Large Files/Locations_of_Greenland_Box_Echograms_S.p','rb'))

midlons = stuff['Midlons']
midlats = stuff['midlats']
files_sorted = stuff['files']

In [ ]:
zee_point = projection.transform_points(ccrs.PlateCarree(),np.asarray(midlons),np.asarray(midlats))

xs = zee_point[:,0]
ys = zee_point[:,1]

In [ ]:
xvals = xs
yvals = ys

points = np.vstack((xvals,yvals)).T

point_tree = spatial.cKDTree(points)

catslen = []

cats = []

catsname = []
catsnamelen = []
fvals = np.asarray(files_sorted)
nearby = []

for num in np.arange(0,len(xvals)):
    cat = point_tree.query_ball_point([xvals[num],yvals[num]],5200.)
    
    
    nearby.append(fvals[cat])

In [ ]:
allnear = []
for nearie in nearby:
    allnear.append(np.unique(nearie))


In [ ]:
nearbys = []
fvals = np.asarray(files_sorted)
for ind,nearby_ind in enumerate(allnear):
    nearby_its = []
    for near in nearby_ind:
        if fvals[ind][35+31:39+31] not in near:
            nearby_its.append(near)
    nearbys.append(nearby_its)

In [ ]:
total_files = []
filelen = []
for ind, nearx in enumerate(nearbys):
    my1file = [files_sorted[ind]]
    
    extra_files = nearx

        
        
    filelen.append(len(extra_files))
    
    for extra_file in extra_files:
        my1file.append(extra_file)
    
    total_files.append(my1file)
    


    
    
sliced_files = total_files

    
    

In [ ]:
pre_finished_files = []
for sliced_file in sliced_files:
    if len(sliced_file)>1:
        pre_finished_files.append(sliced_file)

In [ ]:
pre_finished_files2 = []

for ind,file in enumerate(pre_finished_files):
    first_file = pre_finished_files[ind][0]
    first_year = int(first_file[35+31:39+31])
    first_month = int(first_file[40+31:42+31])
    first_day = int(first_file[43+31:45+31])
    first_date = datetime.datetime(first_year,first_month,first_day)
    
    add_files = [first_file]
    
    for second_file in pre_finished_files[ind][1:]:
        second_year = int(second_file[35+31:39+31])
        second_month = int(second_file[40+31:42+31])
        second_day = int(second_file[43+31:45+31])
        second_date = datetime.datetime(second_year,second_month,second_day)
        if first_date > second_date:
            add_files.append(second_file)

            
    pre_finished_files2.append(add_files)
            



In [ ]:
finished_files = []
for sliced_file in pre_finished_files2:
    if len(sliced_file)>1:
        finished_files.append(sliced_file)



In [ ]:
for ind,finished_file in enumerate(finished_files):
    
    if '2018.04.23' not in finished_file[0]:
        continue
    plt.ioff()
    lining_up_lines(finished_file)
    combining_files(finished_file)

    